In [ ]:
suppressMessages({
library(data.table)
library(dplyr)
library(tidyverse)
library(ggplot2)
library(ChIPseeker)
library(TxDb.Hsapiens.UCSC.hg38.knownGene)
library(clusterProfiler)
library(ReactomePA)
library(ggupset)
library(GenomicRanges)
library(rtracklayer)
library(IRanges)
library(clusterProfiler)
library(DOSE)
library(org.Hs.eg.db)
})


## homer (refseq) vs chipseeker (ucsc) annotations:

####  - ChIPseeker also lets you set the distance you want to consider the TSS for each gene, (-3kb to +3kb) as defined above. HOMER, by default, sets it as -1kb to +100 bp. So you should make sure you're using the same annotations for both - you can change them easily for both HOMER and ChIPseeker - and make sure you have the TSS region set similarly for each. https://www.biostars.org/p/332805/#332807

####  - Regular peak (stringent); not summit!

In [ ]:
#function to save base R plots in multiples formats
save_plot <- function(p, fn, w, h){
    for(ext in c(".pdf", ".png", ".svg")){
        ggsave(filename=paste0(fn,ext), plot=p, width=w, height=h)
    }
}

In [ ]:
txdb <- TxDb.Hsapiens.UCSC.hg38.knownGene

In [ ]:
bed_files <- Sys.glob('../../seacr/control_peaks_nodup_fetal/_m/peakCalling/SEACR/*')
bed_files <- bed_files[grepl('_seacr_control.peaks.stringent.bed',bed_files)]
bed_files <- bed_files[grepl('/SEACR/Br',bed_files)]

head(bed_files)
tail(bed_files)
bed_files

In [ ]:
#bed_files <- Sys.glob('../../seacr/setda1a_atlas-cst_peaks_nodup/_m/peakCalling/SEACR/*')
#bed_files_atlas <- bed_files[grepl('_Atlas.',bed_files)]
#bed_files_atlas <- bed_files_atlas[file.info(bed_files_atlas)$size > 0] #remove any empty file
sample_ids <- gsub('.*/SEACR/|_seacr_.*','',bed_files)
sample_ids <- sample_ids[grepl('^Br',sample_ids)]
sample_ids

In [ ]:
metadata_df <- fread('../../../metadata/_m/metadata_allsamples_CUTTAG_fetal.tsv')
#metadata_df <- subset(metadata_df, Sequence_ID == '2104UNHP-0893')
#metadata_df <- subset(metadata_df, New_Sample_ID %in% sample_ids)
metadata_df
#metadata_df
#metadata_df

In [ ]:
table(metadata_df$markers)

In [ ]:
exp_group <- unique(metadata_df$Group)
exp_group

In [ ]:
# head(peakAnnoSETD1A)
# tail(peakAnnoSETD1A)

In [ ]:
start.time <- Sys.time()

peaksAnnot <- data.frame()
for (i in 1:length(bed_files)){
    df <- fread(bed_files[i], header=F, sep="\t")
    print(paste0(sample_ids[i],' - Number of SEACR Peaks: ',nrow(df)))
    df <- annotatePeak(bed_files[i],TxDb=txdb,tssRegion=c(-1000, 100), verbose=FALSE, annoDb="org.Hs.eg.db")
    df <- as.data.frame(df@anno)
    df$sample_id <- sample_ids[i]
    print(paste0(sample_ids[i],' - Number of Annotated Peaks (chipseeker): ',nrow(df)))
    peaksAnnot <- rbind(df,peaksAnnot)
    }

end.time <- Sys.time()
time.taken <- end.time - start.time
time.taken

In [ ]:
fwrite(peaksAnnot, 'chipseeker_controlpeaks_stringent_annotation_homertss.tsv', sep="\t",quote=F,row.names=F)

In [ ]:
head(peaksAnnot)
tail(peaksAnnot)

In [ ]:
# peaks_annotation.setd1a <- lapply(bed_setd1a[1:2], function(x) {
#                                   df <- annotatePeak(x,tssRegion=c(-3000, 3000), TxDb=txdb, annoDb="org.Hs.eg.db")
#                                   df <- as.data.frame(df@anno)
#                                   df$sample_id <- sample_ids_setd1a[1]
# }
#                )

In [ ]:
# load clusterprofiler  gene list (background universe)

data(geneList)

In [ ]:
dir.create('./cnetplot/')
dir.create('./ora/')

dir.create('./ora/go/')
dir.create('./ora/reactome/')
dir.create('./ora/disgenet/')


path_cnetplot <- './cnetplot/'
path_go <- './ora/go/'
path_reactome <- './ora/reactome/'
path_disgenet <- './ora/disgenet/'






for (i in unique(peaksAnnot$sample_id)){
    
    genes <- subset(peaksAnnot, sample_id == i)$geneId
    
    
    
    #Gene Ontology BP (ORA)
    enrich_go_bp <- enrichGO(gene = genes,
                OrgDb = org.Hs.eg.db,
                ont = "BP",
                pAdjustMethod = "BH",
                pvalueCutoff = 0.05,
                qvalueCutoff = 0.05) 
    
    p1 <- dotplot(enrich_go_bp,showCategory = 25, font.size = 10, label_format = 50) 
    save_plot(p = p1, fn=paste0(path_go, i,'_dotplot_go_bp'),
              h=10,w=8) 
    
    p1 <- barplot(enrich_go_bp,showCategory = 25, font.size = 10, label_format = 50)
    save_plot(p = p1, fn=paste0(path_go, i,'_barplot_go_bp'),
              h=10,w=8)   
    

    #Gene Ontology MF (ORA)
    enrich_go_mf <- enrichGO(gene = genes,
                OrgDb = org.Hs.eg.db,
                ont = "MF",
                pAdjustMethod = "BH",
                pvalueCutoff = 0.05,
                qvalueCutoff = 0.05) 
    
    p1 <- dotplot(enrich_go_mf,showCategory = 25, font.size = 10, label_format = 50) 
    save_plot(p = p1, fn=paste0(path_go, i,'_dotplot_go_mf'),
              h=10,w=8) 
    
    p1 <- barplot(enrich_go_mf,showCategory = 25, font.size = 10, label_format = 50)
    save_plot(p = p1, fn=paste0(path_go, i,'_barplot_go_mf'),
              h=10,w=8)         
    

    #Gene Ontology CC (ORA)
    enrich_go_cc <- enrichGO(gene = genes,
                OrgDb = org.Hs.eg.db,
                ont = "CC",
                pAdjustMethod = "BH",
                pvalueCutoff = 0.05,
                qvalueCutoff = 0.05) 
    
    p1 <- dotplot(enrich_go_cc,showCategory = 25, font.size = 10, label_format = 50) 
    save_plot(p = p1, fn=paste0(path_go, i,'_dotplot_go_cc'),
              h=10,w=8) 
    
    p1 <- barplot(enrich_go_cc,showCategory = 25, font.size = 10, label_format = 50)
    save_plot(p = p1, fn=paste0(path_go, i,'_barplot_go_cc'),
              h=10,w=8)       
    
    
    
    #reactome - ORA 
    enrich_reactome <- enrichPathway(genes)
    
    p1 <- dotplot(enrich_reactome,showCategory = 25, font.size = 10, label_format = 50) 
    save_plot(p = p1, fn=paste0(path_reactome, i,'_dotplot_reactome'),
              h=10,w=8) 
    
    p1 <- barplot(enrich_reactome,showCategory = 25, font.size = 10, label_format = 50)
    save_plot(p = p1, fn=paste0(path_reactome, i,'_barplot_reactome'),
              h=10,w=8)
    
    
    #disgenet - ORA

    enrich_disgenet <- enrichDGN(genes) #disgenet

    p1 <- dotplot(enrich_disgenet,showCategory = 25, font.size = 10, label_format = 50) 
    save_plot(p = p1, fn=paste0(path_disgenet, i,'_dotplot_disgenet'),
              h=10,w=8) 
    
    p1 <- barplot(enrich_disgenet,showCategory = 25, font.size = 10, label_format = 50)
    save_plot(p = p1, fn=paste0(path_disgenet, i,'_barplot_disgenet'),
              h=10,w=8)


    #networks - cnetplot

    edox <- setReadable(enrich_go_bp, 'org.Hs.eg.db', 'ENTREZID')
    p1 <- cnetplot(edox, foldChange=geneList,showCategory = 10, node_label="category", 
                   color_category='firebrick')
    save_plot(p = p1, fn=paste0(path_cnetplot,i,'_cnetplot_go_bp'),
              h=10,w=10)
    
    edox <- setReadable(enrich_go_mf, 'org.Hs.eg.db', 'ENTREZID')
    p1 <- cnetplot(edox, foldChange=geneList,showCategory = 10, node_label="category", 
                   color_category='firebrick')
    save_plot(p = p1, fn=paste0(path_cnetplot,i,'_cnetplot_go_mf'),
              h=10,w=10)
    
        edox <- setReadable(enrich_go_cc, 'org.Hs.eg.db', 'ENTREZID')
    p1 <- cnetplot(edox, foldChange=geneList,showCategory = 10, node_label="category", 
                   color_category='firebrick')
    save_plot(p = p1, fn=paste0(path_cnetplot,i,'_cnetplot_go_cc'),
              h=10,w=10)
    
    edox <- setReadable(enrich_reactome, 'org.Hs.eg.db', 'ENTREZID')
    p1 <- cnetplot(edox, foldChange=geneList,showCategory = 10, node_label="category", 
                   color_category='firebrick')
    save_plot(p = p1, fn=paste0(path_cnetplot,i,'_cnetplot_reactome'),
              h=10,w=10)
    
    edox <- setReadable(enrich_disgenet, 'org.Hs.eg.db', 'ENTREZID')
    p1 <- cnetplot(edox, foldChange=geneList,showCategory = 10, node_label="category", 
                   color_category='firebrick')
    save_plot(p = p1, fn=paste0(path_cnetplot,i,'_cnetplot_disgenet'),
              h=10,w=10)


    }

#genes <- subset(peakAnnoSETD1A, sample_id == unique(peakAnnoSETD1A$sample_id)[1])$geneId

#pathway1 <- enrichPathway(genes)

In [ ]:
# dotplot(enrich_reactome,showCategory = 25, font.size = 10)
# dotplot(enrich_reactome,showCategory = 25, font.size = 10, label_format = 50)
# dotplot(enrich_reactome,showCategory = 25, label_format = 50)

In [ ]:
sessionInfo()